<a href="https://colab.research.google.com/github/drypuddle1220/Sports_Betting_Analytics/blob/main/NBA_predictions_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: read final_df.csv

import pandas as pd

# Assuming 'final_df.csv' is in the current working directory.
# If not, provide the full path to the file.
try:
    df = pd.read_csv('final_df.csv')
    print(df.head()) # Print the first few rows to verify
except FileNotFoundError:
    print("Error: 'final_df.csv' not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")


   SEASON_ID     TEAM_ID TEAM_ABBREVIATION      TEAM_NAME   GAME_ID  \
0      22019  1610612737               ATL  Atlanta Hawks  21900491   
1      22019  1610612737               ATL  Atlanta Hawks  21900477   
2      22019  1610612737               ATL  Atlanta Hawks  21900469   
3      22019  1610612737               ATL  Atlanta Hawks  21900444   
4      22019  1610612737               ATL  Atlanta Hawks  21900431   

    GAME_DATE      MATCHUP WL  MIN  PTS  ...  AST   STL  BLK  TOV  PF  \
0  2019-12-30    ATL @ ORL  W  240  101  ...   21   8.0    4   20  20   
1  2019-12-28    ATL @ CHI  L  239   81  ...   24   8.0    5   19  16   
2  2019-12-27  ATL vs. MIL  L  239   86  ...   20  10.0    8   18  18   
3  2019-12-23    ATL @ CLE  L  239  118  ...   23  11.0    7   16  20   
4  2019-12-21    ATL @ BKN  L  240  112  ...   17   7.0    5   10  22   

   PLUS_MINUS  Total_Pts   injury_pot   avg_pace      agr_off  
0         8.0      210.0   897.697449  45.595952   965.250000  
1     

In [2]:
df.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS', 'Total_Pts',
       'injury_pot', 'avg_pace', 'agr_off'],
      dtype='object')

In [ ]:
from datetime import datetime, timedelta

# Convert 'GAME_DATE' to datetime
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
stat_columns = [ 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']
for col in stat_columns:
    df[f'avg_{col}_20_games'] = 0.0

# Initialize new columns
df['games_past_7_days_team'] = 0

# Calculate games played in the past 7 days for each row
for index, row in df.iterrows():
    team = row['TEAM_NAME']
    game_date = row['GAME_DATE']
    start_date = game_date - timedelta(days=7)

    # Count games for the same team in the past 7 days
    count_7days = df[(df['TEAM_NAME'] == team) &
               (df['GAME_DATE'] >= start_date) &
               (df['GAME_DATE'] < game_date)].shape[0]
    if count_7days < 5:
      df.at[index, 'games_to5'] = 'N'
    else:
      df.at[index, 'games_past_7_days_team'] = 'Y'

    past_20_games = df[(df['TEAM_NAME'] == team) & (df['GAME_DATE'] < game_date)].nlargest(20, 'GAME_DATE')
    wins = past_20_games[past_20_games['WL'] == 'W'].shape[0]
    total_games_20 = past_20_games.shape[0]
    win_loss_ratio = wins / total_games_20 if total_games_20 > 0 else 0
    df.at[index, 'win_loss_ratio_20_games'] = win_loss_ratio

    # Calculate averages for the stat columns
    for col in stat_columns:
        avg_value = past_20_games[col].mean() if not past_20_games.empty else 0
        df.at[index, f'avg_{col}_20_games'] = avg_value

print(df)

<ipython-input-3-6a51f8279a17>:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Y' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'games_past_7_days_team'] = 'Y'


      SEASON_ID     TEAM_ID TEAM_ABBREVIATION          TEAM_NAME   GAME_ID  \
0         22019  1610612737               ATL      Atlanta Hawks  21900491   
1         22019  1610612737               ATL      Atlanta Hawks  21900477   
2         22019  1610612737               ATL      Atlanta Hawks  21900469   
3         22019  1610612737               ATL      Atlanta Hawks  21900444   
4         22019  1610612737               ATL      Atlanta Hawks  21900431   
...         ...         ...               ...                ...       ...   
5807      22017  1610612766               CHA  Charlotte Hornets  21700629   
5808      22017  1610612766               CHA  Charlotte Hornets  21700618   
5809      22017  1610612766               CHA  Charlotte Hornets  21700603   
5810      22017  1610612766               CHA  Charlotte Hornets  21700577   
5811      22017  1610612766               CHA  Charlotte Hornets  21700552   

      GAME_DATE      MATCHUP WL  MIN  PTS  ...  avg_REB_20_game

In [ ]:
df.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS', 'Total_Pts',
       'injury_pot', 'avg_pace', 'agr_off', 'avg_PTS_20_games',
       'avg_FGM_20_games', 'avg_FGA_20_games', 'avg_FG_PCT_20_games',
       'avg_FG3M_20_games', 'avg_FG3A_20_games', 'avg_FG3_PCT_20_games',
       'avg_FTM_20_games', 'avg_FTA_20_games', 'avg_FT_PCT_20_games',
       'avg_OREB_20_games', 'avg_DREB_20_games', 'avg_REB_20_games',
       'avg_AST_20_games', 'avg_STL_20_games', 'avg_BLK_20_games',
       'avg_TOV_20_games', 'avg_PF_20_games', 'avg_PLUS_MINUS_20_games',
       'games_past_7_days_team', 'games_to5', 'win_loss_ratio_20_games'],
      dtype='object')

In [ ]:
# prompt: export df into a csv

from google.colab import files
df.to_csv('exported_df.csv', encoding = 'utf-8-sig')
files.download('exported_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>